# Randomized Benchmarking for Gate Fidelity

This notebook demonstrates randomized benchmarking techniques in LeeQ.

## Contents
- Single-qubit randomized benchmarking
- Two-qubit randomized benchmarking
- Clifford gate sequences
- Fidelity extraction and analysis
- Error rate calculations

## Setup and Imports

In [ ]:
import leeq
import numpy as np
from leeq.experiments.builtin.basic.characterizations.randomized_benchmarking import RandomizedBenchmarkingTwoLevelSubspaceMultilevelSystem
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq.experiments.experiments import ExperimentManager
from leeq.chronicle import Chronicle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import optimize as so
import random

print("✓ LeeQ randomized benchmarking modules loaded successfully")

# Start Chronicle logging
Chronicle().start_log()

# Setup simulation environment optimized for randomized benchmarking
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon with realistic gate errors for RB demonstration
virtual_transmon = VirtualTransmon(
    name="RBQubit",
    qubit_frequency=5040.0,       # 5.04 GHz
    anharmonicity=-200.0,         # -200 MHz
    t1=65.0,                      # 65 μs T1 (good but not perfect)
    t2=40.0,                      # 40 μs T2 (good but not perfect)
    readout_frequency=9500.0,     # 9.5 GHz readout
    quiescent_state_distribution=np.array([0.92, 0.06, 0.02, 0.0])  # Some thermal population
)

# Create simulation setup
setup = HighLevelSimulationSetup(
    name='RandomizedBenchmarkingDemo',
    virtual_qubits={1: virtual_transmon}
)

manager.register_setup(setup)

# Configure qubit for randomized benchmarking (well-calibrated but with some errors)
qubit_config = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.0,
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.503,             # Slightly miscalibrated π-pulse (should be 0.5)
            'phase': 0.02,            # Small phase error
            'width': 0.05,
            'alpha': 495,             # Slightly suboptimal DRAG
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9500.0,
            'channel': 1,
            'shape': 'square',
            'amp': 0.16,
            'phase': 0.0,
            'width': 1.0,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

qubit = TransmonElement(name='Q1', parameters=qubit_config)

# RB-specific measurement settings
from leeq import setup as leeq_setup
leeq_setup().status().set_param("Shot_Number", 1000)  # High statistics for RB
leeq_setup().status().set_param("Shot_Period", 400)   # Efficient repetition

print("✓ Randomized benchmarking setup complete!")
print(f"✓ Qubit configured with realistic gate errors:")
print(f"  - T1: {virtual_transmon.t1:.1f} μs")
print(f"  - T2: {virtual_transmon.t2:.1f} μs")
print(f"  - π-pulse amplitude: {qubit_config['lpb_collections']['f01']['amp']:.3f} (slightly miscalibrated)")
print(f"  - Phase error: {qubit_config['lpb_collections']['f01']['phase']:.3f} rad")
print(f"  - DRAG parameter: {qubit_config['lpb_collections']['f01']['alpha']} (suboptimal)")
print("✓ Ready for randomized benchmarking experiments!")

## Single-Qubit Randomized Benchmarking

**Randomized Benchmarking** (RB) is the gold standard for characterizing gate fidelity in quantum systems. It provides a robust, scalable method to measure average gate performance.

### Theory

RB measures the **average gate fidelity** by applying random sequences of Clifford gates:

1. **Random sequence**: Generate random Clifford gates C₁, C₂, ..., Cₘ
2. **Inversion gate**: Add Cᵢₙᵥ = (Cₘ···C₂C₁)⁻¹ to return to initial state
3. **Measure survival**: Probability of returning to initial state |0⟩
4. **Repeat**: Average over many random sequences

### Key Equation

The survival probability decays exponentially with sequence length:
```
P(m) = A·pᵐ + B
```
where:
- **m**: Sequence length (number of Clifford gates)
- **p**: Depolarizing parameter (related to gate fidelity)  
- **A**: Decay amplitude (~0.5 for perfect initialization/measurement)
- **B**: Asymptotic floor (~0.5 for uniform depolarization)

### Gate Fidelity Extraction

From the fitted decay parameter **p**:
```
Average Gate Fidelity = (d-1)p + 1)/d
```
where **d = 2** for single qubits.

### Why RB Works

- **Clifford gates**: Form a group, enabling perfect inversion sequences
- **Twirling**: Random sequences convert coherent errors to incoherent noise
- **Scalable**: Works for arbitrarily long gate sequences
- **Model-independent**: No assumptions about specific error mechanisms

In [ ]:
# Single-Qubit Randomized Benchmarking Implementation
print("=== Single-Qubit Randomized Benchmarking ===")
print("Measuring average gate fidelity using random Clifford sequences")

# Helper functions for RB analysis
def rb_decay_function(m, A, p, B):
    """RB decay function: P(m) = A * p^m + B"""
    return A * (p ** m) + B

def calculate_gate_fidelity(p, d=2):
    """Convert RB decay parameter to average gate fidelity"""
    return ((d - 1) * p + 1) / d

def simulate_rb_sequence(sequence_length, gate_error_per_gate=0.002):
    """Simulate RB experiment with realistic gate errors"""
    # Each gate has a small error probability
    # For sequence of m gates, survival probability decreases
    
    # Coherent errors (systematic) + incoherent errors (random)
    coherent_error = 0.001  # Systematic amplitude/phase errors
    incoherent_error = gate_error_per_gate - coherent_error
    
    # Total error accumulates with sequence length
    total_error = sequence_length * gate_error_per_gate
    
    # Additional decay from decoherence during long sequences
    # Approximate gate time: 50 ns, T1=65μs, T2=40μs
    gate_time_us = 0.05  # μs per gate
    total_time = sequence_length * gate_time_us
    
    # T1 decay component
    t1_decay = np.exp(-total_time / virtual_transmon.t1)
    
    # T2 dephasing component (more relevant for RB)
    t2_decay = np.exp(-total_time / virtual_transmon.t2)
    
    # Combined survival probability
    gate_survival = 1 - total_error
    decoherence_survival = np.sqrt(t1_decay * t2_decay)  # Geometric mean approximation
    
    # Overall survival probability
    survival_prob = gate_survival * decoherence_survival
    
    return survival_prob

# Define RB sequence lengths (logarithmic spacing for efficiency)
max_sequence_length = 500
sequence_lengths = np.logspace(0, np.log10(max_sequence_length), 15, dtype=int)
sequence_lengths = np.unique(sequence_lengths)  # Remove duplicates
sequence_lengths = sequence_lengths[sequence_lengths <= max_sequence_length]

print(f"\\nRunning RB with sequence lengths: {sequence_lengths}")
print(f"Maximum sequence length: {max_sequence_length} Clifford gates")

# Simulate RB experiment data
print("\\nSimulating randomized benchmarking experiment...")

# Try to use the actual LeeQ RB experiment if available
try:
    # Run LeeQ randomized benchmarking experiment using constructor pattern
    rb_experiment = RandomizedBenchmarkingTwoLevelSubspaceMultilevelSystem(
        dut_list=[qubit],
        kinds=10,                    # Number of random sequences per length
        seq_length=sequence_lengths  # Sequence lengths to test
    )
    
    print("✓ LeeQ RB experiment completed!")
    
    # Extract results from experiment
    if hasattr(rb_experiment, 'result') and rb_experiment.result is not None:
        rb_data = rb_experiment.result
        print("✓ RB data extracted from LeeQ experiment")
    else:
        # Fallback to simulation
        print("Note: Using simulated RB data for analysis")
        rb_data = None
        
except Exception as e:
    print(f"Note: LeeQ RB experiment not available ({e})")
    print("Using high-fidelity simulation for demonstration")
    rb_data = None

# Generate RB data (either from experiment or simulation)
if rb_data is None:
    # Simulate realistic RB data
    survival_probabilities = []
    measurement_errors = []
    
    np.random.seed(42)  # Reproducible results
    
    for seq_len in sequence_lengths:
        # Theoretical survival probability
        theoretical_survival = simulate_rb_sequence(seq_len)
        
        # Add measurement noise (finite sampling statistics)
        measurement_noise = np.random.normal(0, 0.015)  # 1.5% measurement uncertainty
        measured_survival = theoretical_survival + measurement_noise
        
        # Keep in valid probability range
        measured_survival = np.clip(measured_survival, 0, 1)
        
        survival_probabilities.append(measured_survival)
        measurement_errors.append(0.015)  # Constant error bar
    
    survival_probabilities = np.array(survival_probabilities)
    measurement_errors = np.array(measurement_errors)
    
else:
    # Extract from actual experiment
    survival_probabilities = rb_data['survival_probabilities']
    measurement_errors = rb_data.get('errors', np.full_like(survival_probabilities, 0.02))

print(f"✓ RB data generated for {len(sequence_lengths)} sequence lengths")

# Fit RB decay curve
print("\\nFitting randomized benchmarking decay curve...")

# Initial parameter guess
A_guess = 0.5    # Decay amplitude  
p_guess = 0.996  # Depolarizing parameter (high fidelity)
B_guess = 0.5    # Asymptotic floor

initial_guess = [A_guess, p_guess, B_guess]

try:
    # Fit exponential decay
    popt, pcov = so.curve_fit(
        rb_decay_function, 
        sequence_lengths, 
        survival_probabilities,
        p0=initial_guess,
        sigma=measurement_errors,
        absolute_sigma=True,
        bounds=([0, 0, 0], [1, 1, 1])  # Physical bounds
    )
    
    A_fit, p_fit, B_fit = popt
    
    # Calculate parameter uncertainties
    param_errors = np.sqrt(np.diag(pcov))
    A_err, p_err, B_err = param_errors
    
    print(f"✓ RB fit successful!")
    print(f"Fit parameters:")
    print(f"  A (decay amplitude): {A_fit:.4f} ± {A_err:.4f}")
    print(f"  p (depolarizing param): {p_fit:.6f} ± {p_err:.6f}")  
    print(f"  B (asymptotic floor): {B_fit:.4f} ± {B_err:.4f}")
    
except Exception as e:
    print(f"Fit failed: {e}")
    print("Using theoretical parameters for demonstration")
    A_fit, p_fit, B_fit = 0.5, 0.996, 0.5
    A_err, p_err, B_err = 0.01, 0.001, 0.01

# Calculate average gate fidelity
avg_gate_fidelity = calculate_gate_fidelity(p_fit, d=2)
gate_error_rate = 1 - avg_gate_fidelity

# Error propagation for gate fidelity uncertainty
fidelity_error = p_err / 2  # Simplified error propagation

print(f"\\n=== RB Results ===")
print(f"Average gate fidelity: {avg_gate_fidelity:.4f} ± {fidelity_error:.4f}")
print(f"Gate error rate: {gate_error_rate:.6f} ± {fidelity_error:.6f}")
print(f"Gate error rate: {gate_error_rate*1000:.3f} ± {fidelity_error*1000:.3f} × 10⁻³")
print(f"Error rate per gate: {gate_error_rate*100:.4f}%")

# Create RB visualization
print("\\nCreating randomized benchmarking plots...")

# Generate fitted curve for plotting
seq_lengths_fine = np.linspace(0, max(sequence_lengths), 200)
fitted_curve = rb_decay_function(seq_lengths_fine, A_fit, p_fit, B_fit)

# Main RB plot
fig_rb = go.Figure()

# Experimental data with error bars
fig_rb.add_trace(go.Scatter(
    x=sequence_lengths,
    y=survival_probabilities,
    error_y=dict(type='data', array=measurement_errors, visible=True),
    mode='markers',
    name='RB Data',
    marker=dict(size=8, color='blue')
))

# Fitted exponential decay
fig_rb.add_trace(go.Scatter(
    x=seq_lengths_fine,
    y=fitted_curve,
    mode='lines',
    name=f'Exponential Fit (F_avg = {avg_gate_fidelity:.4f})',
    line=dict(color='red', width=3)
))

# Theoretical perfect fidelity (p=1)
perfect_curve = rb_decay_function(seq_lengths_fine, A_fit, 1.0, B_fit)
fig_rb.add_trace(go.Scatter(
    x=seq_lengths_fine,
    y=perfect_curve,
    mode='lines',
    name='Perfect Gates (F = 1.000)',
    line=dict(color='green', dash='dash', width=2)
))

fig_rb.add_annotation(
    x=max(sequence_lengths) * 0.7, y=0.8,
    text=f"P(m) = {A_fit:.3f} × {p_fit:.4f}^m + {B_fit:.3f}<br>" +
         f"Average Gate Fidelity = {avg_gate_fidelity:.4f}<br>" +
         f"Error Rate = {gate_error_rate*1000:.2f} × 10⁻³",
    showarrow=True,
    bgcolor="lightyellow",
    bordercolor="black"
)

fig_rb.update_layout(
    title='Single-Qubit Randomized Benchmarking',
    xaxis_title='Sequence Length (Number of Clifford Gates)',
    yaxis_title='Survival Probability',
    showlegend=True,
    width=800, height=500
)

fig_rb.show()

# Semi-log plot to show exponential decay more clearly
fig_rb_log = go.Figure()

fig_rb_log.add_trace(go.Scatter(
    x=sequence_lengths,
    y=np.log(survival_probabilities - B_fit + 1e-10),  # Subtract floor, avoid log(0)
    error_y=dict(type='data', array=measurement_errors/(survival_probabilities - B_fit + 1e-10), visible=True),
    mode='markers',
    name='Log(Survival - Floor)',
    marker=dict(size=8, color='blue')
))

# Linear fit in log space
log_fitted = np.log(fitted_curve - B_fit + 1e-10)
fig_rb_log.add_trace(go.Scatter(
    x=seq_lengths_fine,
    y=log_fitted,
    mode='lines',
    name=f'Linear Fit (slope = log({p_fit:.4f}) = {np.log(p_fit):.6f})',
    line=dict(color='red', width=3)
))

fig_rb_log.update_layout(
    title='RB Exponential Decay (Semi-log Plot)',
    xaxis_title='Sequence Length',
    yaxis_title='log(Survival Probability - Floor)',
    showlegend=True,
    width=800, height=400
)

fig_rb_log.show()

print("✓ Single-qubit randomized benchmarking complete!")
print(f"✓ Average gate fidelity: {avg_gate_fidelity:.4f}")
print(f"✓ This corresponds to {gate_error_rate*100:.3f}% error per gate operation")

# Performance assessment
print(f"\\n=== Performance Assessment ===")
if avg_gate_fidelity > 0.999:
    print("🏆 Excellent gate fidelity (>99.9%) - Suitable for quantum error correction")
elif avg_gate_fidelity > 0.995:
    print("✅ Very good gate fidelity (>99.5%) - Suitable for NISQ algorithms")
elif avg_gate_fidelity > 0.99:
    print("⚠️  Good gate fidelity (>99.0%) - May need optimization for complex algorithms")
else:
    print("❌ Gate fidelity needs improvement - Check calibration and noise sources")

## Two-Qubit Randomized Benchmarking

**Two-qubit randomized benchmarking** extends RB to characterize multi-qubit gate performance, which is crucial for quantum computing scalability.

### Key Differences from Single-Qubit RB

1. **Gate Set**: Uses two-qubit Clifford group (much larger than single-qubit)
2. **Complexity**: 11,520 two-qubit Cliffords vs 24 single-qubit Cliffords
3. **Fidelity Formula**: Average gate fidelity = (3p + 1)/4 for two qubits (d=4)
4. **Cross-talk**: Measures correlated errors between qubits

### Applications

- **CNOT gate characterization**: Most common two-qubit gate
- **Entangling gate fidelity**: Critical for quantum algorithms  
- **Crosstalk quantification**: Unwanted qubit-qubit interactions
- **Scalability assessment**: Can we maintain fidelity with more qubits?

### Challenges

- **Long sequences**: Two-qubit gates are slower and noisier
- **State preparation**: Requires high-fidelity initialization of both qubits
- **Readout**: Simultaneous measurement of correlated qubit states
- **Clifford compilation**: Complex gate decomposition into native gates

### Typical Results

- **Best two-qubit gates**: 99%+ fidelity (superconducting qubits)
- **Good two-qubit gates**: 95-99% fidelity (adequate for NISQ)  
- **Error budget**: Two-qubit gates usually dominate total error
- **Decoherence**: Longer gates → more T1/T2 decay

In [ ]:
# Two-Qubit Randomized Benchmarking Simulation
print("=== Two-Qubit Randomized Benchmarking ===")  
print("Simulating two-qubit gate fidelity measurement")

# Note: This is a simulation since two-qubit RB requires a more complex setup
print("\\nNote: This demonstrates the analysis approach for two-qubit RB")
print("In practice, this requires two coupled qubits and CNOT gate implementation")

def simulate_two_qubit_rb(sequence_length, two_qubit_error_rate=0.01):
    """Simulate two-qubit RB with realistic parameters"""
    
    # Two-qubit gates are typically ~10x worse than single-qubit gates
    single_qubit_error = 0.002
    two_qubit_error = two_qubit_error_rate
    
    # Typical two-qubit Clifford has ~1.5 CNOTs + several single-qubit gates  
    cnots_per_clifford = 1.5
    single_gates_per_clifford = 4
    
    # Total error per Clifford
    error_per_clifford = (cnots_per_clifford * two_qubit_error + 
                          single_gates_per_clifford * single_qubit_error)
    
    # Two-qubit gate time is typically 10-20x longer than single-qubit
    gate_time_us = 0.5  # μs per two-qubit Clifford
    total_time = sequence_length * gate_time_us
    
    # Decoherence during longer two-qubit sequences
    t1_decay = np.exp(-total_time / virtual_transmon.t1)
    t2_decay = np.exp(-total_time / virtual_transmon.t2)
    decoherence_survival = np.sqrt(t1_decay * t2_decay)
    
    # Gate error survival
    gate_survival = (1 - error_per_clifford) ** sequence_length
    
    # Combined survival
    total_survival = gate_survival * decoherence_survival
    
    return total_survival

# Simulate two-qubit RB data
print("\\nSimulating two-qubit randomized benchmarking...")

# Shorter sequences due to higher error rates
two_qubit_seq_lengths = np.array([1, 2, 4, 8, 16, 32, 64, 128, 200])
two_qubit_survivals = []
two_qubit_errors = []

# Different error rates for comparison
error_rates = [0.005, 0.01, 0.02]  # 0.5%, 1%, 2% two-qubit gate error
colors = ['green', 'orange', 'red']
labels = ['Excellent (0.5%)', 'Good (1.0%)', 'Fair (2.0%)']

fig_two_qubit = go.Figure()

for i, error_rate in enumerate(error_rates):
    survivals = []
    np.random.seed(42 + i)  # Different seed for each error rate
    
    for seq_len in two_qubit_seq_lengths:
        theoretical = simulate_two_qubit_rb(seq_len, error_rate)
        measured = theoretical + np.random.normal(0, 0.02)  # Measurement noise
        measured = np.clip(measured, 0, 1)
        survivals.append(measured)
    
    survivals = np.array(survivals)
    
    # Fit decay curve
    try:
        popt, _ = so.curve_fit(
            rb_decay_function, 
            two_qubit_seq_lengths, 
            survivals,
            p0=[0.75, 0.98, 0.25],  # Different initial guess for two-qubit
            bounds=([0, 0, 0], [1, 1, 1])
        )
        
        A_fit, p_fit, B_fit = popt
        
        # Calculate two-qubit gate fidelity (d=4 for two qubits)
        two_qubit_fidelity = (3 * p_fit + 1) / 4
        
    except:
        # Fallback values
        p_fit = 1 - error_rate
        two_qubit_fidelity = (3 * p_fit + 1) / 4
        A_fit, B_fit = 0.75, 0.25
    
    # Plot data points
    fig_two_qubit.add_trace(go.Scatter(
        x=two_qubit_seq_lengths,
        y=survivals,
        mode='markers',
        name=f'{labels[i]} - F = {two_qubit_fidelity:.3f}',
        marker=dict(size=8, color=colors[i])
    ))
    
    # Plot fitted curves
    seq_fine = np.linspace(0, max(two_qubit_seq_lengths), 100)
    fitted = rb_decay_function(seq_fine, A_fit, p_fit, B_fit)
    
    fig_two_qubit.add_trace(go.Scatter(
        x=seq_fine,
        y=fitted,
        mode='lines',
        name=f'{labels[i]} Fit',
        line=dict(color=colors[i], width=2),
        showlegend=False
    ))

fig_two_qubit.update_layout(
    title='Two-Qubit Randomized Benchmarking (Simulation)',
    xaxis_title='Sequence Length (Two-Qubit Cliffords)',
    yaxis_title='Survival Probability',
    showlegend=True,
    width=800, height=500
)

fig_two_qubit.show()

# Compare single-qubit vs two-qubit performance
print("\\n=== Single-Qubit vs Two-Qubit Comparison ===")

comparison_data = {
    'Gate Type': ['Single-Qubit', 'Two-Qubit (Excellent)', 'Two-Qubit (Good)', 'Two-Qubit (Fair)'],
    'Error Rate (%)': [0.2, 0.5, 1.0, 2.0],
    'Gate Fidelity': [avg_gate_fidelity, 0.995, 0.990, 0.980],
    'Typical Gate Time (μs)': [0.05, 0.2, 0.3, 0.5]
}

# Create comparison visualization
fig_comparison = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Gate Fidelity Comparison', 'Error Rate Comparison']
)

# Fidelity comparison
fig_comparison.add_trace(
    go.Bar(x=comparison_data['Gate Type'], 
           y=comparison_data['Gate Fidelity'],
           name='Fidelity',
           marker_color=['blue', 'green', 'orange', 'red']),
    row=1, col=1
)

# Error rate comparison (log scale)
fig_comparison.add_trace(
    go.Bar(x=comparison_data['Gate Type'],
           y=comparison_data['Error Rate (%)'],
           name='Error Rate (%)',
           marker_color=['blue', 'green', 'orange', 'red']),
    row=1, col=2
)

fig_comparison.update_yaxes(title_text="Gate Fidelity", row=1, col=1, range=[0.95, 1.0])
fig_comparison.update_yaxes(title_text="Error Rate (%)", row=1, col=2, type="log")
fig_comparison.update_layout(title='Single-Qubit vs Two-Qubit Gate Performance', height=400, showlegend=False)
fig_comparison.show()

# Print comparison table
print("\\nGate Performance Summary:")
print("-" * 70)
print(f"{'Gate Type':<20} {'Fidelity':<10} {'Error Rate':<12} {'Gate Time':<12}")
print("-" * 70)
for i, gate_type in enumerate(comparison_data['Gate Type']):
    fidelity = comparison_data['Gate Fidelity'][i]
    error_rate = comparison_data['Error Rate (%)'][i]
    gate_time = comparison_data['Typical Gate Time (μs)'][i]
    print(f"{gate_type:<20} {fidelity:<10.4f} {error_rate:<12.2f}% {gate_time:<12.2f} μs")

# Circuit depth analysis
print("\\n=== Quantum Circuit Depth Analysis ===")
print("Maximum useful circuit depth before >50% error probability:")

for i, gate_type in enumerate(comparison_data['Gate Type']):
    error_rate = comparison_data['Error Rate (%)'][i] / 100
    max_gates = int(np.log(0.5) / np.log(1 - error_rate))
    print(f"{gate_type:<20}: ~{max_gates:<3} gates")

print("\\n=== Key Insights ===")
print("✓ Two-qubit gates typically have 5-50x higher error rates than single-qubit gates")
print("✓ Two-qubit gate fidelity is the limiting factor for circuit depth")
print("✓ CNOT gates require careful calibration and error mitigation")
print("✓ Crosstalk between qubits becomes significant in multi-qubit systems")
print("✓ Two-qubit RB is essential for characterizing entangling operations")

print("\\n=== Practical Recommendations ===") 
print("• Target >99% two-qubit gate fidelity for quantum error correction")
print("• Optimize CNOT decomposition to minimize gate count")
print("• Use simultaneous randomized benchmarking for crosstalk detection")
print("• Monitor two-qubit fidelity drift over time")
print("• Consider alternative entangling gates (iSWAP, CZ) if CNOT fidelity is low")

print("\\n✓ Two-qubit randomized benchmarking analysis complete!")
print("✓ This framework extends to characterize any multi-qubit gate set")
print("✓ Essential for scaling quantum computing to larger systems")

## Summary and Applications

### What We Accomplished

This notebook demonstrated the complete randomized benchmarking toolkit for gate fidelity assessment:

1. **Single-Qubit Randomized Benchmarking**
   - Measured average gate fidelity using exponential decay fitting
   - Achieved reliable fidelity measurements with error quantification
   - Demonstrated scalable approach for gate characterization
   - Compared performance against theoretical benchmarks

2. **Two-Qubit Randomized Benchmarking**
   - Simulated multi-qubit gate fidelity measurement challenges
   - Compared single-qubit vs two-qubit gate performance
   - Analyzed circuit depth limitations from gate errors
   - Provided practical recommendations for improvement

### Key Results

- **Single-qubit fidelity**: >99% achievable with good calibration
- **Two-qubit challenge**: 5-50× higher error rates than single-qubit gates
- **Scalability limits**: Two-qubit gates determine maximum circuit depth
- **Model independence**: RB works regardless of specific error mechanisms

### Why Randomized Benchmarking Matters

**Compared to other characterization methods:**

| Method | Advantages | Limitations |
|--------|------------|-------------|
| **Process Tomography** | Complete gate characterization | Exponential scaling, not scalable |
| **Randomized Benchmarking** | Scalable, model-independent | Average fidelity only, no error details |
| **Gate Set Tomography** | Full gate set + SPAM errors | Complex analysis, computationally intensive |

### Practical Applications

**Device Optimization**
- Monitor gate fidelity drift over time
- Compare different calibration protocols  
- Optimize gate parameters for maximum fidelity
- Benchmark competing quantum technologies

**Algorithm Design**
- Determine maximum useful circuit depth
- Guide error mitigation strategies
- Select optimal gate decompositions
- Assess feasibility of quantum algorithms

**Quality Control**
- Production testing of quantum devices
- Certification for quantum computing systems
- Performance guarantees for end users
- Continuous monitoring in production

### Next Steps

- **Custom Experiments**: Build specialized characterization protocols
- **Multi-qubit scaling**: Extend to >2 qubits with crosstalk analysis
- **Real-time optimization**: Use RB feedback for dynamic calibration
- **Advanced protocols**: Cycle benchmarking, cross-entropy benchmarking

### Continue Learning

Continue to [custom_experiments.ipynb](custom_experiments.ipynb) to learn how to create specialized experimental protocols building on these characterization techniques.